# 一、处理回单

## 1.1 拆分PDF

### 1.1.1 招商银行

#### 其他银行的处理方式与招商银行大同小异，为节省篇幅以及更简洁地介绍项目，在此仅以招商银行为例

In [1]:
# 模块导入
import os
import fitz  # PyMuPDF
from pathlib import Path

In [66]:
# 定义文件夹路径
input_folder = Path(r"D:\视图\凭证整理自动化项目\1. input\回单")
output_folder = Path(r"D:\视图\凭证整理自动化项目\2. process\回单\回单拆分")

# 确保输出文件夹存在
output_folder.mkdir(parents=True, exist_ok=True)

# 遍历文件夹中的所有 PDF 文件
i = 1
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.pdf'):
        # 构造完整的文件路径
        input_path = input_folder / filename
        output_path = output_folder / f"{filename[:-4]}_拆分后的回单.pdf"  # 添加后缀以区分
        
        # 打开 PDF 文件
        pdf_document = fitz.open(input_path)
        
        # 创建新的 PDF 文档
        new_document = fitz.open()
        
        # 遍历每一页
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            text = page.get_text("text")  # 使用 "text" 模式提取文本
            #print(f"文件 {filename} 第 {page_num + 1} 页提取的文本：\n{text}\n")  # 调试输出
            
            if "税(费)种名称" in text:
                # 如果包含“税(费)种名称”，则完整保留该页面
                new_document.insert_pdf(pdf_document, from_page=page_num, to_page=page_num)
                print(f"文件 {filename} 第 {page_num + 1} 页包含 '税(费)种名称'，完整保留。")
            else:
                # 如果不包含“税(费)种名称”，则分割为上下两部分
                rect = page.rect
                mid_y = rect.height / 2
                top_half = fitz.Rect(rect.x0, rect.y0, rect.x1, mid_y)  # 上半部分矩形
                bottom_half = fitz.Rect(rect.x0, mid_y, rect.x1, rect.y1)  # 下半部分矩形
                
                # 将上半部分插入新文档
                new_page_top = new_document.new_page(width=rect.width, height=mid_y)
                new_page_top.show_pdf_page(new_page_top.rect, pdf_document, page_num, clip=top_half)
                
                # 将下半部分插入新文档
                new_page_bottom = new_document.new_page(width=rect.width, height=mid_y)
                new_page_bottom.show_pdf_page(new_page_bottom.rect, pdf_document, page_num, clip=bottom_half)
                #print(f"文件 {filename} 第 {page_num + 1} 页不包含 '税(费)种名称'，分割为上下两部分。")
        
        # 保存分割后的 PDF
        new_document.save(output_path)
        new_document.close()
        print(f"\n{i}、已处理文件：{filename} -> {output_path}")
        i = i + 1

print("所有文件处理完成。")


1、已处理文件：A公司.pdf -> D:\视图\凭证整理自动化项目\2. process\回单\回单拆分\A公司_拆分后的回单.pdf

2、已处理文件：B公司.pdf -> D:\视图\凭证整理自动化项目\2. process\回单\回单拆分\B公司_拆分后的回单.pdf
所有文件处理完成。


## 1.2 提取PDF信息

### 1.2.1 招商银行

In [13]:
# 导入模块
import os
import fitz  # PyMuPDF
import pandas as pd
import re
from pathlib import Path

In [67]:
# 定义公司主体列表
company_list = [
    "A公司", "B公司"
]

# 定义输入输出路径
input_folder = Path(r"D:\视图\凭证整理自动化项目\2. process\回单\回单拆分")
output_path = Path(r"D:\视图\凭证整理自动化项目\2. process\回单\回单信息提取.xlsx")

# 初始化一个空的DataFrame用于存储提取的信息
columns = ["文件名", "公司主体", "账单类型", "交易日期", "交易金额", "页码"]
data = []

# 遍历文件夹中的所有PDF文件
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.pdf'):
        pdf_path = input_folder / filename
        
        # 打开PDF文件
        pdf_document = fitz.open(pdf_path)
        
        # 遍历每一页
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            text = page.get_text()
            
            # 判断是否包含“税(费)种名称”
            is_tax_page = "税(费)种名称" in text
            
            # 提取账单类型
            bill_type = None
            if "出账回单" in text:
                bill_type = "出 账 回 单"
            elif "入账回单" in text:
                bill_type = "入 账 回 单"
            
            # 提取公司主体
            company = None
            if bill_type == "入 账 回 单":
                company_match = re.search(r"收款人：([^\s]+)", text)  # 匹配收款人后的非空白字符
            elif bill_type == "出 账 回 单":
                company_match = re.search(r"付款人：([^\s]+)", text)  # 匹配付款人后的非空白字符
            
            if company_match:
                company = company_match.group(1).strip()  # 提取并去除多余空格
                # 检查是否在公司主体列表中
                if company not in company_list:
                    company = None  # 如果不在列表中，则忽略
            
            # 如果仍未提取到公司主体，则检查公司主体列表
            if not company:
                for company_name in company_list:
                    if company_name in text:
                        company = company_name
                        break
            
            # 提取交易日期
            date_pattern = r"\d{4}年\d{1,2}月\d{1,2}日"
            transaction_date = re.search(date_pattern, text)
            transaction_date = transaction_date.group() if transaction_date else None
            
            # 提取交易金额
            if is_tax_page:
                # 如果包含“税(费)种名称”，按照“交易金额(小写)： ￥3,717.84”格式提取
                amount_pattern = r"交易金额\(小写\)\：\s*￥([\d\.,]+)"
            else:
                # 否则按照原来的格式提取
                amount_pattern = r"交易金额\(小写\)\：CNY([\d\.,]+)"
            
            transaction_amount = re.search(amount_pattern, text)
            if transaction_amount:
                amount_str = transaction_amount.group(1).replace(",", "")  # 移除逗号
                try:
                    transaction_amount = float(amount_str)  # 转换为浮点数
                except ValueError:
                    transaction_amount = None  # 如果转换失败，设置为None
            else:
                transaction_amount = None
            
            # 将提取的信息存储到列表中
            data.append([filename, company, bill_type, transaction_date, transaction_amount, page_num + 1])
        
        pdf_document.close()

# 将提取的数据保存到Excel表格中
df_zsyh = pd.DataFrame(data, columns=columns)

# 按照公司主体、账单类型、交易日期、交易金额进行分组
df_zsyh['分组标记'] = df_zsyh.groupby(['公司主体', '账单类型', '交易日期', '交易金额']).cumcount() + 1
df_zsyh['银行'] = '招商银行'

df_hdtq = pd.concat([df_zsyh],axis=0, ignore_index=True)

df_hdtq.to_excel(output_path, index=False)

print(f"提取完成，信息已保存到：{output_path}")

提取完成，信息已保存到：D:\视图\凭证整理自动化项目\2. process\回单\回单信息提取.xlsx


# 二、处理会计凭证

## 2.1 凭证分组标记

#### 将会计凭证导出为excel，利用python统一格式，并进行分组标记

In [5]:
# 导入模块
import pandas as pd
from pathlib import Path

In [68]:
# 定义输入输出文件路径
input_path = Path(r"D:\视图\凭证整理自动化项目\1. input\凭证\凭证.xlsx")
output_path = Path(r"D:\视图\凭证整理自动化项目\2. process\凭证\凭证-分组标记.xlsx")


# 读取Excel文件
df = pd.read_excel(input_path, sheet_name='Sheet1', header=0)  # 第一行作为表头

# 确保“原币金额”列为数值类型
df['原币金额'] = pd.to_numeric(df['原币金额'], errors='coerce')

# 确保“银行账号”列为文本类型
df['银行账号'] = df['银行账号'].astype(str)

# 新增“交易日期”列，赋值为“摘要”列从左往右的10个字符
df['交易日期'] = df['摘要'].astype(str).str[:10]

# 新增“类型”列
def assign_type(summary):
    summary = str(summary)  # 确保summary是字符串
    if "收到" in summary[:12]:  # 如果前12个字符包含“收到”
        return "入 账 回 单"
    elif "支付" in summary[:13]:  # 如果前13个字符包含“支付”
        return "出 账 回 单"
    elif "收到" in summary:  # 如果摘要中包含“收到”
        return "入 账 回 单"
    elif "支付" in summary:  # 如果摘要中包含“支付”
        return "出 账 回 单"
    else:
        return ""  # 默认值为空

df['类型'] = df['摘要'].apply(assign_type)

# 将“交易日期”列格式转换为“yyyy年mm月dd日”，并确保为文本类型
df['交易日期'] = pd.to_datetime(df['交易日期'], errors='coerce').dt.strftime('%Y年%m月%d日')

# 去掉“账簿”列中的空格
df['账簿'] = df['账簿'].astype(str).str.strip()

# 选择需要的列
df = df[['账簿', '凭证号', '交易日期', '类型', '摘要', '科目编码', '科目全名', '币别', '原币金额', '银行账号', '银行']]

# 按照账簿、交易日期、类型、原币金额、银行列进行分组
grouped = df.groupby(['账簿', '交易日期', '类型', '原币金额','银行'], dropna=False)

# 为每组数据生成从1开始的标记
df['分组标记'] = grouped.cumcount() + 1

# 保存处理后的数据到新的Excel文件
df.to_excel(output_path, index=False)

print(f"处理完成，修改后的文件已保存到：{output_path}")

处理完成，修改后的文件已保存到：D:\视图\凭证整理自动化项目\2. process\凭证\凭证-分组标记.xlsx


## 2.2 凭证拆分

#### 将会计凭证导出为PDF，按公司与凭证号分组，利用python拆分凭证

In [17]:
# 导入模块
import os
import re
import fitz  # PyMuPDF
import pandas as pd

In [69]:
# 定义输入和输出路径
input_pdf_path = r"D:\视图\凭证整理自动化项目\1. input\凭证\凭证.pdf"
output_folder = r"D:\视图\凭证整理自动化项目\2. process\凭证\凭证拆分"

def extract_text_from_page(page):
    """从PDF页面中提取文本"""
    text = page.get_text()
    return text

def extract_company_and_voucher(text):
    """从文本中提取公司主体和凭证号"""
    company_pattern = r"核算单位 Unit\s*(.+)"
    voucher_pattern = r"编号：记\s*(\d+)"
    
    company_match = re.search(company_pattern, text)
    voucher_match = re.search(voucher_pattern, text)
    
    company = company_match.group(1).strip() if company_match else None
    voucher = voucher_match.group(1).strip() if voucher_match else None
    
    return company, voucher

def split_and_save_pdf(input_pdf_path, output_folder):
    """拆分PDF并保存"""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 打开PDF文件
    doc = fitz.open(input_pdf_path)
    results = []  # 用于存储提取结果
    page_groups = {}  # 用于分组存储页面

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = extract_text_from_page(page)
        company, voucher = extract_company_and_voucher(text)
        
        if company and voucher:
            group_key = f"{company}_{voucher}"
            if group_key not in page_groups:
                page_groups[group_key] = []
            page_groups[group_key].append(page_num + 1)  # 页码从1开始
            results.append([company, voucher, page_num + 1])
    
    # 保存分组后的PDF文件
    for group_key, pages in page_groups.items():
        output_pdf_path = os.path.join(output_folder, f"{group_key}号凭证.pdf")
        new_doc = fitz.open()  # 创建一个新的PDF文档
        for page_num in pages:
            new_doc.insert_pdf(doc, from_page=page_num - 1, to_page=page_num - 1)
        new_doc.save(output_pdf_path)
        new_doc.close()
    
    doc.close()
    
    # 保存提取结果到Excel文件
    df = pd.DataFrame(results, columns=["公司主体", "凭证号", "PDF页码"])
    #excel_path = os.path.join(output_folder, "凭证-提取结果.xlsx")
    #df.to_excel(excel_path, index=False, engine="openpyxl")
    #print(f"提取结果已保存到：{excel_path}")

# 执行拆分和保存操作
split_and_save_pdf(input_pdf_path, output_folder)
print(f"拆分后的凭证已保存到：{output_folder}")

拆分后的凭证已保存到：D:\视图\凭证整理自动化项目\2. process\凭证\凭证拆分


# 三、回单与凭证匹配

## 3.1 为回单匹配正确凭证号

In [70]:
import pandas as pd
from pathlib import Path

# 定义文件路径
pdf_data_path = Path(r"D:\视图\凭证整理自动化项目\2. process\回单\回单信息提取.xlsx")  # 回单PDF提取信息
excel_data_path = Path(r"D:\视图\凭证整理自动化项目\2. process\凭证\凭证-分组标记.xlsx")  # 处理后的Excel数据
output_path1 = Path(r"D:\视图\凭证整理自动化项目\2. process\凭证&回单-匹配\凭证-匹配结果.xlsx")  # 输出文件
output_path2 = Path(r"D:\视图\凭证整理自动化项目\2. process\凭证&回单-匹配\回单信息提取-匹配结果.xlsx")  # 输出文件

# 读取PDF提取信息
pdf_df = pd.read_excel(pdf_data_path)

# 读取处理后的Excel数据
excel_df = pd.read_excel(excel_data_path)

# 初始化匹配结果列
pdf_df['匹配结果'] = "未匹配"
pdf_df['凭证号'] = None  # 用于记录匹配的凭证号
excel_df['匹配结果'] = "未匹配"
excel_df['匹配的PDF页码'] = None  # 用于记录匹配的PDF页码


In [71]:
# 使用 merge 进行匹配
# 匹配更新回单提取信息表--------------
merged_df_pdf = pd.merge(
    pdf_df,
    excel_df,
    left_on=["公司主体", "账单类型", "交易日期", "交易金额", "分组标记","银行"],
    right_on=["账簿", "类型", "交易日期", "原币金额", "分组标记","银行"],
    how="left",
    suffixes=('_pdf', '_excel')  # 为重复列添加后缀
)
# 更新匹配结果
pdf_df['匹配结果'] = merged_df_pdf['凭证号_excel'].notna().replace({True: "已匹配", False: "未匹配"})
pdf_df['凭证号'] = merged_df_pdf['凭证号_excel']

# 匹配更新excel表-------------------
# 使用 merge 进行匹配
merged_df_excel = pd.merge(
    pdf_df,
    excel_df,
    left_on=["公司主体", "账单类型", "交易日期", "交易金额", "分组标记","银行"],
    right_on=["账簿", "类型", "交易日期", "原币金额", "分组标记","银行"],
    how="right",
    suffixes=('_pdf', '_excel')  # 为重复列添加后缀
)
excel_df['匹配结果'] = merged_df_excel['文件名'].notna().replace({True: "已匹配", False: "未匹配"})
excel_df['匹配的PDF页码'] = merged_df_excel['页码']


In [72]:
# 保存匹配结果
pdf_df.to_excel(output_path2, index=False)  # 保存PDF匹配结果
excel_df.to_excel(output_path1, index=False)  # 保存Excel匹配结果

print(f"匹配完成，结果已保存到：{output_path1}\n匹配完成，结果已保存到：{output_path2}")

匹配完成，结果已保存到：D:\视图\凭证整理自动化项目\2. process\凭证&回单-匹配\凭证-匹配结果.xlsx
匹配完成，结果已保存到：D:\视图\凭证整理自动化项目\2. process\凭证&回单-匹配\回单信息提取-匹配结果.xlsx


## 3.2 按凭证号合并输出匹配后的回单PDF

### 3.2.1 招商银行

In [73]:
#导入模块
import os
import fitz  # PyMuPDF
import pandas as pd
from pathlib import Path

In [76]:
# 定义输入输出路径
input_excel = Path(r"D:\视图\凭证整理自动化项目\2. process\凭证&回单-匹配\回单信息提取-匹配结果.xlsx")
input_pdf_folder = Path(r"D:\视图\凭证整理自动化项目\2. process\回单\回单拆分")
output_folder = Path(r"D:\视图\凭证整理自动化项目\2. process\凭证&回单-匹配")

# 确保输出文件夹存在
output_folder.mkdir(parents=True, exist_ok=True)

# 读取回单提取信息的Excel文件
df = pd.read_excel(input_excel)
df = df[df['银行']=='招商银行']

# 按公司主体和凭证号分组
grouped = df.groupby(["公司主体", "凭证号"])

# 遍历每个分组
for (company, voucher), group in grouped:
    # 创建一个新的PDF文档（用于合并普通页面）
    output_pdf = fitz.open()
    
    # 创建一个新的PDF文档（用于单独保存大页面）
    large_pdf = fitz.open()
    
    # 按页码升序排序
    group = group.sort_values(by="页码")
    
    # 每页最多放6个PDF页面
    pages_per_sheet = 6
    num_pages = len(group)
    num_sheets = (num_pages + pages_per_sheet - 1) // pages_per_sheet  # 总页数
    
    for sheet in range(num_sheets):
        # 创建一个新的空白页面（A4纵向）
        new_page = output_pdf.new_page(width=595, height=842)  # A4纵向页面大小
        start_idx = sheet * pages_per_sheet
        end_idx = min(start_idx + pages_per_sheet, num_pages)
        
        for i, (_, row) in enumerate(group.iloc[start_idx:end_idx].iterrows()):
            # 获取PDF文件路径
            pdf_path = input_pdf_folder / row["文件名"]
            if not pdf_path.exists():
                print(f"警告：文件 {pdf_path} 不存在，跳过。")
                continue
            
            try:
                # 打开PDF文件
                pdf_document = fitz.open(pdf_path)
                if pdf_document is None:
                    print(f"警告：无法打开文件 {pdf_path}，跳过。")
                    continue
                
                # 获取对应页码的页面
                page_num = int(row["页码"]) - 1  # 页码从0开始
                if page_num < 0 or page_num >= len(pdf_document):
                    print(f"警告：页码 {page_num + 1} 超出范围，文件 {pdf_path} 跳过。")
                    pdf_document.close()
                    continue
                
                page = pdf_document.load_page(page_num)
                
                # 判断页面高度是否超过500
                if page.rect.height > 500:
                    # 如果高度超过500，单独保存到大页面PDF
                    large_pdf.insert_pdf(pdf_document, from_page=page_num, to_page=page_num)
                    pdf_document.close()
                    continue  # 跳过当前循环，不与其他PDF合并
                
                # 计算放置位置（纵向布局，每页6个PDF，2列3行）
                col = i % 2  # 列索引（0或1）
                row_idx = i // 2  # 行索引（0、1或2）
                x = col * 297.5  # 水平位置
                y = row_idx * 280  # 垂直位置（适当调整间距）
                
                # 计算缩放比例
                target_width = 297.5  # 目标宽度
                target_height = 280  # 目标高度
                scale_width = target_width / page.rect.width
                scale_height = target_height / page.rect.height
                scale = min(scale_width, scale_height)  # 取较小的缩放比例
                
                # 定义目标矩形区域
                target_rect = fitz.Rect(x, y, x + target_width, y + target_height)
                
                # 将PDF页面插入到新页面，并缩放
                new_page.show_pdf_page(target_rect, pdf_document, page_num)
                
                # 在左上角添加凭证号（竖直显示，字号为24）
                voucher_num = str(int(row["凭证号"]))  # 确保凭证号为整型
                rect = fitz.Rect(x + 250, y + 70, x + 290, y + 280)  # 右上角矩形区域，调整大小以适应24号字体
                new_page.insert_text(rect.tl, voucher_num, fontname="helv", fontsize=24, color=(0, 0, 0), rotate=0)  # 竖直显示
                
                pdf_document.close()
            except Exception as e:
                print(f"错误：处理文件 {pdf_path}第{page_num + 1}页时发生异常：{e}\n")
                continue
    
    # 保存合并后的PDF文件（普通页面）
    if len(output_pdf) > 0:
        output_filename = f"{company}_{int(voucher)}号凭证_招商银行.pdf"
        output_path = output_folder / output_filename
        output_pdf.save(output_path)
        output_pdf.close()
        #print(f"已保存合并后的PDF文件：{output_path}")
    else:
        print(f"警告：{company}_{int(voucher)} 没有有效的普通页面，跳过保存。")
    
    # 保存单独的大页面PDF文件
    if len(large_pdf) > 0:
        large_output_filename = f"{company}_{int(voucher)}号凭证_招商银行_税费.pdf"
        large_output_path = output_folder / large_output_filename
        large_pdf.save(large_output_path)
        large_pdf.close()
        print(f"已保存税费PDF文件：{large_output_path}")
    #else:
        #print(f"警告：{company}_{int(voucher)} 没有大页面，跳过保存。")

print(f"所有PDF文件已按公司主体和凭证号分组合并，并保存到：{output_folder}")

所有PDF文件已按公司主体和凭证号分组合并，并保存到：D:\视图\凭证整理自动化项目\2. process\凭证&回单-匹配


## 3.3 凭证与回单交叉合并打印

In [78]:
import os
import fitz  # PyMuPDF
from pathlib import Path
import re

# 定义多个源文件夹路径
source_folders = [
    Path(r"D:\视图\凭证整理自动化项目\2. process\凭证\凭证拆分"),
    Path(r"D:\视图\凭证整理自动化项目\2. process\凭证&回单-匹配"),
]

# 定义目标文件夹路径
target_folder = Path(r"D:\视图\凭证整理自动化项目\3. output")

# 确保目标文件夹存在
target_folder.mkdir(parents=True, exist_ok=True)

# 支持的文件类型
supported_extensions = ['.pdf', '.jpg', '.jpeg', '.png', '.bmp']

# 自然排序函数
def natural_sort_key(s):
    """
    将字符串中的数字部分转换为整数，用于自然排序。
    例如："公司1_12_银行" -> ["公司", 1, "_", 12, "_银行"]
    """
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', str(s))]

# 按公司主体分组文件
company_files = {}

# 遍历每个源文件夹
for source_folder in source_folders:
    if not source_folder.is_dir():  # 确保源文件夹存在
        print(f"警告：源文件夹 {source_folder} 不存在，跳过。")
        continue

    print(f"正在处理源文件夹：{source_folder}")
    
    # 遍历源文件夹中的所有文件
    for file in source_folder.iterdir():
        if file.is_file() and file.suffix.lower() in supported_extensions:
            # 提取文件名中第一个“_”前的文本作为公司主体
            company_name = file.name.split("_")[0]

            # 将文件按公司主体分组
            if company_name not in company_files:
                company_files[company_name] = []
            company_files[company_name].append(file)

# 合并每个公司主体的文件
for company_name, files in company_files.items():
    # 按文件名自然排序
    files.sort(key=lambda x: natural_sort_key(x.name))

    # 创建一个新的PDF文档
    merged_pdf = fitz.open()

    # 遍历文件并合并
    for file in files:
        if file.suffix.lower() == '.pdf':
            # 如果是PDF文件，直接插入
            pdf_document = fitz.open(file)
            merged_pdf.insert_pdf(pdf_document)
            pdf_document.close()
        else:
            # 如果是图片文件，插入到新页面
            imgdoc = fitz.open(file)  # 打开图片文件
            rect = imgdoc[0].rect  # 获取图片的矩形区域
            pdfbytes = imgdoc.convert_to_pdf()  # 将图片转换为PDF
            imgpdf = fitz.open("pdf", pdfbytes)  # 打开转换后的PDF
            page = merged_pdf.new_page(width=rect.width, height=rect.height)  # 创建新页面
            page.show_pdf_page(rect, imgpdf, 0)  # 将图片插入到新页面
            imgpdf.close()

    # 保存合并后的PDF文件
    output_filename = f"{company_name}_整理后的会计凭证.pdf"
    output_path = target_folder / output_filename
    merged_pdf.save(output_path)
    merged_pdf.close()
    print(f"已合并并保存文件：{output_path}")

print("所有文件合并完成")

正在处理源文件夹：D:\视图\凭证整理自动化项目\2. process\凭证\凭证拆分
正在处理源文件夹：D:\视图\凭证整理自动化项目\2. process\凭证&回单-匹配
已合并并保存文件：D:\视图\凭证整理自动化项目\3. output\A公司_整理后的会计凭证.pdf
已合并并保存文件：D:\视图\凭证整理自动化项目\3. output\B公司_整理后的会计凭证.pdf
所有文件合并完成
